# BeamNN Eval A/B Testing UI Generation

Later on, we can choose to use jsPsych instead of Google Sheets

In [1]:
# for each model
# for each scan_battery
# for each target
# per target, we rank, e.g.,

# [,
#   model_1_target_1_cnn, (beter than cnn)
#   target_1_dnn_benchmark
#   model_2_target_1_cnn, 
#   target_1_das_benchmark,
#   model_3_target_1_cnn (worse than cnn),
# ]

# so we need 5*4/2 = 10 unordered 2-combination comparisons

# (model_1_target_1_cnn, target_1_dnn_benchmark)
# (model_1_target_1_cnn, model_2_target_1_cnn)
# (model_1_target_1_cnn, target_1_das_benchmark)
# (model_1_target_1_cnn, model_3_target_1_cnn)
# (target_1_dnn_benchmark, model_2_target_1_cnn)
# (target_1_dnn_benchmark, target_1_das_benchmark)
# (target_1_dnn_benchmark, model_3_target_1_cnn)
# (model_2_target_1_cnn, target_1_das_benchmark)
# (model_2_target_1_cnn, model_3_target_1_cnn)
# (target_1_das_benchmark, model_3_target_1_cnn)

# At presentation time, the order of the two can be reversed at random.

# 

# 1. Generate 2 combinations

In [2]:
# 1. For each target, grab all models cnns, and append a. dnn benchmark, b. das benchmark

In [43]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os
import glob

import matplotlib.pyplot as plt
# import cv2
from cv2 import imread
import pandas as pd
from PIL import Image

from analysis_utils import get_df, inspect_model_by_name, compare_two_models

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import warnings
warnings.filterwarnings('error')
warnings.filterwarnings("ignore", category=DeprecationWarning)

identifier = '*'
models_df = get_df(identifier)

In [5]:
models_df_sample_50 = models_df.sample(n=50)

In [49]:
SCAN_BATTERIES_DIRNAME = '../scan_batteries'

# each row represent one target

columns = [
    'target_name', # identify the target
    'scan_battery_name', # identify the target
    'model_name',
    'das_benchmark',
    'dnn_benchmark',
    # Then there are many dynamically added columns that represent each model, such as 
    # 'model_1_cnn',
    # 'model_2_cnn',
]

# TODO: Two-Column (scan_battery_name, target_name) indexing because target numbers are not unique.

images_per_target_df = pd.DataFrame(columns=columns)

# images_per_target_df['target_name'].astype(str) # useless
images_per_target_df.set_index('target_name', inplace=True)


# Build the 
for scan_battery_dirname in glob.glob(os.path.join(SCAN_BATTERIES_DIRNAME, "*/")):
#     print()
    scan_battery_name = os.path.basename(os.path.dirname(scan_battery_dirname))
#     print('scan_battery_dirname =', scan_battery_dirname)
#     print('scan_battery_name =', scan_battery_name)
    target_path = os.path.join(scan_battery_dirname, "target*")
#     print('target_path =', target_path)
    for target_dir in glob.glob(target_path):
#         print('target_dir =', target_dir)
        target_name = os.path.basename(target_dir)
#         print('target_name =', target_name)
        images_per_target_df.loc[target_name, 'scan_battery_name'] = scan_battery_name
        for model_index, model_row in models_df_sample_50.iterrows():
#             print(model_index)
            model_cnn_image_colname = model_name + '_cnn_image'
            if model_cnn_image_colname not in images_per_target_df:
                
                images_per_target_df[model_cnn_image_colname] = pd.Series(dtype=object)
#                 print('add {} as a column'.format(model_cnn_image_colname))
#                 print(images_per_target_df.dtypes)
            model_name = model_row['name']
            cnn_image_fname_abs = os.path.abspath(os.path.join('..', 'DNNs', model_name, 'scan_batteries', scan_battery_name, target_name, 'dnn.png'))
#             print('cnn_image_fname_abs =', cnn_image_fname_abs)
#             images_per_target_df.loc[target_name, 'cnn_file'] = images_per_target.name.map(lambda name: f'../DNNs/{name}/{scan}/dnn.jpg')

#             images_per_target_df.loc[target_name, model_name + '_cnn_image_fname'] = cnn_image_fname
            images_per_target_df.at[target_name, 'model_name'] = model_name
#             image = imread(cnn_image_fname_abs)
            image = Image.open(cnn_image_fname_abs)
            print('type(image) =', type(image))
            images_per_target_df.loc[target_name, model_cnn_image_colname] = image
        
        # Problem is that if we use this approach, we'd have to compare all models.
        
        

add 20180709112043_14_evaluated_cnn_image as a column
scan_battery_name                        object
model_name                               object
das_benchmark                            object
dnn_benchmark                            object
20180709112043_14_evaluated_cnn_image    object
dtype: object
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
add 20180718164411_8_evaluated_cnn_image as a column
scan_battery_name                        object
model_name                               object
das_benchmark                            object
dnn_benchmark                            object
20180709112043_14_evaluated_cnn_image    object
20180718164411_8_evaluated_cnn_image     object
dtype: object
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
add 20181016023040_13_evaluated_cnn_image as a column
scan_battery_name                        object
model_name                               object
das_benchmark      

add 20181016023040_40_evaluated_cnn_image as a column
scan_battery_name                         object
model_name                                object
das_benchmark                             object
dnn_benchmark                             object
20180709112043_14_evaluated_cnn_image     object
20180718164411_8_evaluated_cnn_image      object
20181016023040_13_evaluated_cnn_image     object
20180718164411_48_evaluated_cnn_image     object
20181025215535_459_evaluated_cnn_image    object
20180718164411_31_evaluated_cnn_image     object
20181025215535_204_evaluated_cnn_image    object
20181025215535_316_evaluated_cnn_image    object
20180718164411_4_evaluated_cnn_image      object
20180709112043_30_evaluated_cnn_image     object
20181025215535_385_evaluated_cnn_image    object
20181025215535_271_evaluated_cnn_image    object
20181025215535_107_evaluated_cnn_image    object
20181025215535_343_evaluated_cnn_image    object
20180718164411_11_evaluated_cnn_image     object
20181021132748_

add 20181025215535_352_evaluated_cnn_image as a column
scan_battery_name                         object
model_name                                object
das_benchmark                             object
dnn_benchmark                             object
20180709112043_14_evaluated_cnn_image     object
20180718164411_8_evaluated_cnn_image      object
20181016023040_13_evaluated_cnn_image     object
20180718164411_48_evaluated_cnn_image     object
20181025215535_459_evaluated_cnn_image    object
20180718164411_31_evaluated_cnn_image     object
20181025215535_204_evaluated_cnn_image    object
20181025215535_316_evaluated_cnn_image    object
20180718164411_4_evaluated_cnn_image      object
20180709112043_30_evaluated_cnn_image     object
20181025215535_385_evaluated_cnn_image    object
20181025215535_271_evaluated_cnn_image    object
20181025215535_107_evaluated_cnn_image    object
20181025215535_343_evaluated_cnn_image    object
20180718164411_11_evaluated_cnn_image     object
20181021132748

add 20181025215535_201_evaluated_cnn_image as a column
scan_battery_name                         object
model_name                                object
das_benchmark                             object
dnn_benchmark                             object
20180709112043_14_evaluated_cnn_image     object
20180718164411_8_evaluated_cnn_image      object
20181016023040_13_evaluated_cnn_image     object
20180718164411_48_evaluated_cnn_image     object
20181025215535_459_evaluated_cnn_image    object
20180718164411_31_evaluated_cnn_image     object
20181025215535_204_evaluated_cnn_image    object
20181025215535_316_evaluated_cnn_image    object
20180718164411_4_evaluated_cnn_image      object
20180709112043_30_evaluated_cnn_image     object
20181025215535_385_evaluated_cnn_image    object
20181025215535_271_evaluated_cnn_image    object
20181025215535_107_evaluated_cnn_image    object
20181025215535_343_evaluated_cnn_image    object
20180718164411_11_evaluated_cnn_image     object
20181021132748

add 20181021132748_53_evaluated_cnn_image as a column
scan_battery_name                         object
model_name                                object
das_benchmark                             object
dnn_benchmark                             object
20180709112043_14_evaluated_cnn_image     object
20180718164411_8_evaluated_cnn_image      object
20181016023040_13_evaluated_cnn_image     object
20180718164411_48_evaluated_cnn_image     object
20181025215535_459_evaluated_cnn_image    object
20180718164411_31_evaluated_cnn_image     object
20181025215535_204_evaluated_cnn_image    object
20181025215535_316_evaluated_cnn_image    object
20180718164411_4_evaluated_cnn_image      object
20180709112043_30_evaluated_cnn_image     object
20181025215535_385_evaluated_cnn_image    object
20181025215535_271_evaluated_cnn_image    object
20181025215535_107_evaluated_cnn_image    object
20181025215535_343_evaluated_cnn_image    object
20180718164411_11_evaluated_cnn_image     object
20181021132748_

add 20181025215535_289_evaluated_cnn_image as a column
scan_battery_name                         object
model_name                                object
das_benchmark                             object
dnn_benchmark                             object
20180709112043_14_evaluated_cnn_image     object
20180718164411_8_evaluated_cnn_image      object
20181016023040_13_evaluated_cnn_image     object
20180718164411_48_evaluated_cnn_image     object
20181025215535_459_evaluated_cnn_image    object
20180718164411_31_evaluated_cnn_image     object
20181025215535_204_evaluated_cnn_image    object
20181025215535_316_evaluated_cnn_image    object
20180718164411_4_evaluated_cnn_image      object
20180709112043_30_evaluated_cnn_image     object
20181025215535_385_evaluated_cnn_image    object
20181025215535_271_evaluated_cnn_image    object
20181025215535_107_evaluated_cnn_image    object
20181025215535_343_evaluated_cnn_image    object
20180718164411_11_evaluated_cnn_image     object
20181021132748

type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImag

type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImag

type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImag

type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImageFile'>
type(image) = <class 'PIL.PngImagePlugin.PngImag

In [34]:
images_per_target_df

,scan_battery_name,model_name,das_benchmark,dnn_benchmark,20180709112043_14_evaluated_cnn_image,20180718164411_8_evaluated_cnn_image,20181016023040_13_evaluated_cnn_image,20180718164411_48_evaluated_cnn_image,20181025215535_459_evaluated_cnn_image,20180718164411_31_evaluated_cnn_image,...,20181002150342_37_evaluated_cnn_image,20180928201821_31_evaluated_cnn_image,20181025215535_168_evaluated_cnn_image,20181002150342_25_evaluated_cnn_image,20180706214952_26_evaluated_cnn_image,20181025215535_289_evaluated_cnn_image,20180718164411_24_evaluated_cnn_image,20180928201821_14_evaluated_cnn_image,20180709112043_5_evaluated_cnn_image,20181025215535_91_evaluated_cnn_image
target_name,,,,,,,,,,,,,,,,,,,,,
target_17,target_in_vivo,20181025215535_91_evaluated,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
target_19,target_in_vivo,20181025215535_91_evaluated,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
target_5_SNR_10dB,target_anechoic_cyst_5mm,20181025215535_91_evaluated,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
target_4_SNR_10dB,target_anechoic_cyst_5mm,20181025215535_91_evaluated,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
target_2_SNR_10dB,target_anechoic_cyst_5mm,20181025215535_91_evaluated,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
target_3_SNR_10dB,target_anechoic_cyst_5mm,20181025215535_91_evaluated,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
target_1_SNR_10dB,target_anechoic_cyst_5mm,20181025215535_91_evaluated,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
target_5,target_phantom_anechoic_cyst_2p5mm,20181025215535_91_evaluated,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
target_2,target_phantom_anechoic_cyst_2p5mm,20181025215535_91_evaluated,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
images_per_target_df.dtypes

scan_battery_name                         object
model_name                                object
das_benchmark                             object
dnn_benchmark                             object
20180709112043_14_evaluated_cnn_image     object
20180718164411_8_evaluated_cnn_image      object
20181016023040_13_evaluated_cnn_image     object
20180718164411_48_evaluated_cnn_image     object
20181025215535_459_evaluated_cnn_image    object
20180718164411_31_evaluated_cnn_image     object
20181025215535_204_evaluated_cnn_image    object
20181025215535_316_evaluated_cnn_image    object
20180718164411_4_evaluated_cnn_image      object
20180709112043_30_evaluated_cnn_image     object
20181025215535_385_evaluated_cnn_image    object
20181025215535_271_evaluated_cnn_image    object
20181025215535_107_evaluated_cnn_image    object
20181025215535_343_evaluated_cnn_image    object
20180718164411_11_evaluated_cnn_image     object
20181021132748_1_evaluated_cnn_image      object
20181016023040_40_ev